# Подключаем яндекс.диск - Смотрите в консоль:
> Предложит войти на ya.ru/device и ввести код из консоли для разрешения доступа к диску! 
> Внимание! Будет синхронизовано всё содержимое яндекс.диска. 
> Используйте чистый акк!

In [ ]:
#@markdown # Смотрите в консоль, она предложит войти на ya.ru/device и ввести код из консоли для разрешения доступа к диску! Внимание! Будет синхронизовано всё содержимое яндекс.диска. Используйте чистый акк!
from IPython.display import clear_output
from IPython.utils import capture
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - оставьте пустым, если не хотите использовать яндекс.диск

print("[0;33mУстанавливаем клиент Яндекс.Диска...")

!sudo wget https://repo.yandex.ru/yandex-disk/yandex-disk_latest_amd64.deb -O /tmp/yandex-disk.deb && sudo dpkg -i /tmp/yandex-disk.deb
!sudo apt-get install yandex-disk
clear_output()
print("[0;33mАвторизуем клиент Яндекс.Диска...")
!yandex-disk token
!yandex-disk start --dir=/kaggle/working/drive

if Shared_Drive!="" and os.path.exists("/kaggle/working/drive/sd"):
  mainpth="drive"
else:
  mainpth="drive"

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /kaggle/working/$mainpth/sd
  %cd /kaggle/working/$mainpth/sd
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
  !mkdir -p /kaggle/working/$mainpth/sd/stable-diffusion-webui/cache/
  os.environ['TRANSFORMERS_CACHE']=f"/kaggle/working/{mainpth}/sd/stable-diffusion-webui/cache"
    
!conda create -n webui_env -c cctbx202208 -y
!source /opt/conda/bin/activate webui_env && conda install -q -c cctbx202208 python -y

clear_output()
inf('\u2714 Успешно!','success', '50px')

# Предварительный запуск и установка зависимостей:

In [ ]:
#@markdown # Для установки зависимостей, после успешного старта можно остановить и подгрузить модели
%cd /kaggle/working/drive/sd/stable-diffusion-webui    
!/opt/conda/envs/webui_env/bin/python3 launch.py --share --xformers --api --theme=dark

# Скачивание Моделей с huggingface/civitai:

In [ ]:
import os
import requests
from tqdm.auto import tqdm
#@markdown # Скачивание Моделей с huggingface/civitai
# Тут укажите ссылку на модель
URL = "https://civitai.com/api/download/models/15236" #@param {type:"string"} 
# Тут укажите тип загружаемой модели, варианты перечислены в скобках []
Directory = "Model" #@param ["Model", "VAE", "Lora", "Hypernetworks", "Textual Inversions"]
#@markdown - Если нужно в другое место укажи путь.
path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/" #@param {type:"string"}
VAE_predownload = "vae-ft-mse-840000-ema-pruned.safetensors" #@param ["none", "vae-ft-mse-840000-ema-pruned.safetensors"]

if path:
  destination_path = path
else:
  if Directory == "Model":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/"
  elif Directory == "VAE":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/"
  elif Directory == "Lora":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/Lora/"
  elif Directory == "Hypernetworks":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/hypernetworks/"
  elif Directory == "Textual Inversions":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/embeddings/"

if VAE_predownload == "vae-ft-mse-840000-ema-pruned.safetensors":
  if os.path.exists('/kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors'):
      inf('\u2714 Done VAE уже есть в папке','success', '50px')
  else:
      inf('\u2718 Загружаем VAE с https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors','success', "250px")
      %cd /kaggle/working/
      !wget -O /kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
      inf('\u2714 VAE успешно загружено!','success', '200px')
    
if not URL:
    print("Вы не ввели ссылку для скачивания.")
else:
    if "civitai" in URL:
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024 # 1 Kibibyte
        progress_bar_length = 512
        progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True, ncols=progress_bar_length)
        filename = response.headers.get('content-disposition')
        if filename:
            filename = filename.split('filename=')[1].strip('\"')
        else:
            filename = URL.split("/")[-1]
        destination_file_path = os.path.join(destination_path, filename)

        # Create the directory if it does not exist
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)

        with open(destination_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    f.write(chunk)
                    progress_bar.update(len(chunk))
        progress_bar.close()

    elif "huggingface" in URL:
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024 # 1 Kibibyte
        progress_bar_length = 512
        progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True, ncols=progress_bar_length)

        filename = URL.split("/")[-1]
        destination_file_path = os.path.join(destination_path, filename)

        # Create the directory if it does not exist
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)

        with open(destination_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    f.write(chunk)
                    progress_bar.update(len(chunk))
        progress_bar.close()

    if path:
        print(f"Файл {filename} загрузился в папку для {path}.")
    else:
        print(f"Файл {filename} загрузился в папку для {Directory}.")

# Запуск Stable-diffusion-webui: 

In [ ]:
%cd /kaggle/working/drive/sd/stable-diffusion-webui    

!/opt/conda/envs/webui_env/bin/python3 launch.py --share --xformers --api --theme=dark